# Environment

In [1]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb, chromsizes Tileset
import clodius
import os
import os.path as path
import pandas as pd
import itertools

import negspy.coordinates as nc

print(os.getcwd())
print(clodius.__version__)
print(higlass.__version__)

/c/Users/james/Documents/Davos/Results/2019-10-24_higlass
0.14.2
0.4.0


## Load annotations

In [9]:
genes = beddb("Data/hg38/gene-annotations-hg38.beddb")
chrom_sizes = chromsizes("hg38.chrom.sizes")

chrom_labels = {
    "top": Track(
        track_type="horizontal-chromosome-labels",
        tileset=chrom_sizes,
        position="top",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-chromosome-labels",
        tileset=chrom_sizes,
        position="left",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    )
}

gene_annots = {
    "top": Track(
        track_type="horizontal-gene-annotations",
        tileset=genes,
        position="top",
        height=150,
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-gene-annotations",
        tileset=genes,
        position="left",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    )
}

ctcf_motifs = {
    "top": Track(
        track_type="bedlike",
        server="//higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="top",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="bedlike",
        server="//higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="left",
        options={
            "fontSize": 12,
            "showMousePosition": True,
        },
    )
}

hg38 = nc.get_chrominfo("hg38")

# Data
## Contact matrices

In [3]:
metadata = pd.read_csv(path.join("..", "..", "Data", "External", "LowC_Samples_Data_Available.tsv"), sep="\t", header=0)
tumour_samples = ["PCa" + str(i) for i in metadata["Sample ID"].tolist()]

benign_metadata = pd.read_csv(path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "config.tsv"), sep="\t", header=0)
benign_metadata = benign_metadata.loc[~benign_metadata["Sample"].str.startswith("PCa"), :]
benign_samples = benign_metadata["Sample"].tolist()

all_samples = tumour_samples + benign_samples

cooler_files = (
    [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in tumour_samples]
    + [path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "Contacts", s + ".mcool") for s in benign_samples]
)
lowc_tilesets = {s: cooler(f) for s, f in zip(all_samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

min_resolution = 5000

lowc_heatmaps = {
    s: {
        "top": Track(
            track_type='horizontal-heatmap',
            position='top',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            },
        ),
        "bottom": Track(
            track_type='horizontal-heatmap',
            position='bottom',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        ),
        "left": Track(
            track_type='vertical-heatmap',
            position='left',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        ),
        "right": Track(
            track_type='vertical-heatmap',
            position='right',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        ),
        "centre": Track(
            track_type='heatmap',
            position='center',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=500,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
            }
        )
    } for s in all_samples
}

## TADs

In [4]:
windows = list(range(3, 31))

tad_1D_files = {s: {w: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "separated-TADs", s + ".40000bp.w_" + str(w) + ".domains.beddb") for w in windows} for s in tumour_samples}
tad_2D_files = {s: {w: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "separated-TADs", s + ".40000bp.w_" + str(w) + ".domains.bed2ddb") for w in windows} for s in tumour_samples}
htad_files = {s: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bed2ddb") for s in tumour_samples}

def bed2ddb(filepath, uuid=None, **kwargs):
    from clodius.tiles.utils import tiles_wrapper_2d
    from clodius.tiles.bed2ddb import get_2d_tileset_info, get_2D_tiles
    return Tileset(
        uuid=uuid,
        tileset_info=lambda: get_2d_tileset_info(filepath),
        tiles=lambda tids: tiles_wrapper_2d(
            tids,
            lambda z,x,y: get_2D_tiles(filepath, z, x, y)[(x, y)]
        ),
        **kwargs
    )

tad_1D_tilesets = {s: {w: beddb(v[w], name=s + " TADs (w=" + str(w) +")") for w in windows} for s, v in tad_1D_files.items()}
tad_2D_tilesets = {s: {w: bed2ddb(v[w], name=s + " TADs (w=" + str(w) +")") for w in windows} for s, v in tad_2D_files.items()}
htad_tilesets = {s: bed2ddb(v, name=s + " TADs") for s, v in htad_files.items()}

tads = {
    s: {
        w: {
           "top": Track(
                track_type="bedlike",
                position='top',
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "bottom": Track(
                track_type="bedlike",
                position='bottom',
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "left": Track(
                track_type="bedlike",
                position="left",
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "right": Track(
                track_type="bedlike",
                position="right",
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                },
            ),
        } for w in windows
    } for s in tumour_samples
}

tad_polygons = {
    s: {
        w: {
           "top": Track(
                track_type="horizontal-2d-rectangle-domains",
                position="top",
                tileset=tad_2D_tilesets[s][w],
                filetype="bed2ddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "bottom": Track(
                track_type="horizontal-2d-rectangle-domains",
                position='bottom',
                tileset=tad_2D_tilesets[s][w],
                filetype="bed2ddb",
                options={
                    "showMousePosition": True,
                },
            ),
            "centre": Track(
                track_type="2d-rectangle-domains",
                position="center",
                tileset=tad_2D_tilesets[s][w],
                filetype="bed2ddb",
            ),
        } for w in windows
    } for s in tumour_samples
}

hierarchical_tads = {
    s: {
       "top": Track(
            track_type="2d-rectangle-domains",
            position="top",
            tileset=htad_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
            },
        ),
        "bottom": Track(
            track_type="2d-rectangle-domains",
            position="bottom",
            tileset=htad_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
            },
        ),
        "centre": Track(
            track_type="2d-rectangle-domains",
            position="center",
            tileset=htad_tilesets[s],
            filetype="bed2ddb",
        ),
    } for s in tumour_samples
}

# Images
## Generic

In [12]:
display, server, viewconf = higlass.display([
    View(
        tracks=[chrom_labels["top"], gene_annots["top"]],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr14", 36000000, hg38),
            nc.chr_pos_to_genome_pos("chr14", 39000000, hg38)
        ],
    )
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'RTl7S9u4RJmntzEDlL1y1Q', 'tracks': {'top': [{'ty…

## FOXA1

In [59]:
foxa1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]]
        + [CombinedTrack([lowc_heatmaps["PCa13848"]["centre"], tad_polygons["PCa13848"][10]["centre"]])]
    ,
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 36000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 39000000, hg38)
    ],
)

display, server, viewconf = higlass.display([foxa1_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'UpOh2PUsSa2MjZgqafCrPg', 'tracks': {'top': [{'ty…

## _TMPRSS2_-_ERG_ fusion

In [50]:
t2e_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]]
        + [lowc_heatmaps["PCa" + str(s)]["top"] for s in metadata.loc[metadata["T2E Status"] == "Yes", "Sample ID"]]
        + [lowc_heatmaps["PCa" + str(s)]["top"] for s in metadata.loc[metadata["T2E Status"] == "No", "Sample ID"]]
        + [lowc_heatmaps[s]["top"] for s in benign_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr21", 30000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 48000000, hg38)
    ],
)

display, server, viewconf = higlass.display([
    t2e_view
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'GOr9ero3SG67R0uFxRs1Bw', 'tracks': {'top': [{'ty…

/Users/hawleyj/miniconda3/lib/python3.7/site-packages/clodius/tiles/format.py:29: RuntimeWarning: All-NaN slice encountered
  min_dense = float(np.nanmin(data))
/Users/hawleyj/miniconda3/lib/python3.7/site-packages/clodius/tiles/format.py:28: RuntimeWarning: All-NaN slice encountered
  max_dense = float(np.nanmax(data))
/Users/hawleyj/miniconda3/lib/python3.7/site-packages/clodius/tiles/format.py:29: RuntimeWarning: All-NaN slice encountered
  min_dense = float(np.nanmin(data))


## T2E translocation

In [14]:
t2e_translocation = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["bottom"],
        lowc_heatmaps["PCa13848"]["centre"],
        lowc_heatmaps["PCa13848"]["left"],
        #hierarchical_tads["PCa13848"]
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)
nont2e_translocation = View(
    tracks=[
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["top"],
        lowc_heatmaps["PCa3023"]["centre"],
        lowc_heatmaps["PCa3023"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38500000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 41500000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)

t2e_translocation_insertion = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["centre"],
        tads["PCa13848"][3]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 34000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37000000, hg38)
    ],
)
nont2e_translocation_insertion = View(
    tracks=[
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["centre"],
        tads["PCa3023"][3]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 34000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37000000, hg38)
    ],
)


display, server, viewconf = higlass.display([t2e_translocation, nont2e_translocation])
#display, server, viewconf = higlass.display([t2e_translocation_insertion, nont2e_translocation_insertion])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'd4baBJkDThuwLZZZSBA4hQ', 'tracks': {'top': [{'ty…

## AR

In [25]:
ar_tracks = [chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples]
ar_view = View(
    tracks=ar_tracks,
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 65040000, hg38),
        nc.chr_pos_to_genome_pos("chrX", 70200000, hg38)
    ]
)

display, server, viewconf = higlass.display([ar_view], value_scale_syncs = [ar_tracks])
display
#help(higlass.display)

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'TzSTcQXQRj2UQ2I3EEU9_A', 'tracks': {'top': [{'ty…

## MYC

In [26]:
myc_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr8", 126000000, hg38),
        nc.chr_pos_to_genome_pos("chr8", 130000000, hg38)
    ],
)

display, server, viewconf = higlass.display([myc_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'XH0d2aESRy-cN8R_T3wNZg', 'tracks': {'top': [{'ty…

## CTCF, YY1, HOXB13

In [27]:
ctcf_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr16", 65000000, hg38),
        nc.chr_pos_to_genome_pos("chr16", 69000000, hg38)
    ],
)
yy1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 98000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 102000000, hg38)
    ],
)
hoxb13_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr17", 46160000, hg38),
        nc.chr_pos_to_genome_pos("chr17", 51320000, hg38)
    ],
)

myc_insertion_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr12", 70480000 - 5e6, hg38),
        nc.chr_pos_to_genome_pos("chr12", 70570000 + 5e6, hg38)
    ],
)

display, server, viewconf = higlass.display([
    myc_insertion_view
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'RxagWVS_RWSg3QWOGkUSbw', 'tracks': {'top': [{'ty…

## Miscellaneous

In [42]:
test_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [val for pair in zip([lowc_heatmaps[s]["top"] for s in tumour_samples], [tads[s][3]["top"] for s in tumour_samples]) for val in pair],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr4", 175000000, hg38),
        nc.chr_pos_to_genome_pos("chr4", 190000000, hg38)
    ],
)

test_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"], lowc_heatmaps["PCa13848"]["top"], lowc_heatmaps["PCa3023"]["top"]] + [tads[s][10]["top"] for s in tumour_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr10", 96320000, hg38),
        nc.chr_pos_to_genome_pos("chr10", 97400000, hg38)
    ],
)


display, server, viewconf = higlass.display([test_view])
display

#"genomePositionSearchBoxVisible": true,
#      "genomePositionSearchBox": {
#        "autocompleteServer": "http://higlass.io/api/v1",
#        "chromInfoServer": "http://higlass.io/api/v1",
#        "visible": true,
#        "chromInfoId": "hg38",
#        "autocompleteId": "P0PLbQMwTYGy-5uPIQid7A"
#      }
#help(higlass.display)

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'V-62tF99Q52f0bKFxy7_KQ', 'tracks': {'top': [{'ty…